In [1]:
##control.do
#!pip install linearmodels
#!pip install econtools
import pandas as pd
import numpy as np
import matplotlib as plt
from linearmodels import PanelOLS
import statsmodels.api as sm
import econtools as econ
import econtools.metrics as mt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1

(16127, 31)


In [3]:
for i in range(len(auth_list)):
    name = 'trend_pa_'+str(i+1)
    df[name] = 0
    for j in range(len(df)):
        if df.loc[j, 'id_auth'] == i+1 :
            df.loc[j, name] = df.loc[j,'trend']
        if df.loc[j, 'authority_code']==3090272 or df.loc[j, 'authority_code']==3070001:
            df.loc[j, name] = 0
#for statement check            
#df.head

In [4]:
work_dum = pd.get_dummies(df['work_category'])
year_dum = pd.get_dummies(df['year'])
work_list = list(work_dum.columns)
year_list = list(year_dum.columns)

df_dum = pd.concat([year_dum, work_dum],axis = 1)
df = pd.concat([df, df_dum],axis = 1)

# table 4 odd columns

In [ ]:
#table 4
#(:,[1,3,5,7])/ co& ctrl_exp
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']
treatment = ['turin_co_sample','turin_pr_sample']
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration for group
for o in outcome:
    for t in treatment:
        df_reg_t = df[(df[t]==1)&(df['ctrl_exp_' + t]==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
        idx = df_reg_t[df_reg_t[o].isnull()==True].index
        df_reg_o = df_reg_t.drop(idx)
        #vif cal
        #first, make a column list
        reg_col = []
        for i in work_list:
            reg_col.append(i)
        for j in year_list:
            reg_col.append(j)
        exog_var = ['fpsb_auction','reserve_price','municipality','fiscal_efficiency'] #id_auth빼봄
        exog = exog_var + reg_col 


        #check multicollinearity
        X = df_reg_o.loc[:,exog]
        vif = calc_vif(X)
        #print(vif)


        #delete from col list
        for i in range(len(vif)):
            if np.isnan(vif.loc[i, 'VIF']) == True:
                reg_col.remove(vif.loc[i, 'variables'])
            elif vif.loc[i,'VIF'] > 10:
                for j in exog_var:
                    if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                        exog_var.remove(vif.loc[i,'variables'])
                
        exog = exog_var + reg_col
        #exog.remove('id_auth')
        exog.remove(2000)
        exog.remove('OG01')
        exog.remove('municipality')

        #print(exog) #check
        print(mt.reg(df_reg_o, o, exog, fe_name = 'authority_code', cluster = 'auth_anno'))

# Table 4 even columns

In [14]:
#table 4 discount
#(:,2)/ co/pr& ctrl_exp #trend_pa 들어가서 무지하게 큼

treatment = ['turin_co_sample','turin_pr_sample']
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration for group
for t in treatment:
    df_reg_co = df[(df[t]==1)&(df['ctrl_exp_' + t]==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
    
    #vif cal
    #first, make a column list
    reg_col = ['trend_pa_2','trend_pa_3','trend_pa_5','trend_pa_8','trend_pa_15','trend_pa_19','trend_pa_24','trend_pa_29','trend_pa_33','trend_pa_585','trend_pa_1231',
              'trend_pa_1480', 'trend_pa_1690','trend_pa_1721','trend_pa_1749','trend_pa_1839']
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','reserve_price','municipality','trend','trend_treat'] #id_auth빼봄
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_reg_co.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    #exog.remove('id_auth')
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')

    #print(exog) #check
    fe_reg = mt.reg(df_reg_co, 'discount', exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(fe_reg)

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			1262
R-squared:		0.6497
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  101
Fixed effects by:	authority_code
  No. of FE:		  15
               coeff    se       t   p>t  CI_low CI_high
fpsb_auction   5.941 1.381   4.302 0.000   3.201   8.681
reserve_price  0.000 0.000   6.110 0.000   0.000   0.000
trend_treat    2.264 0.416   5.440 0.000   1.438   3.089
trend_pa_2     0.126 1.452   0.087 0.931  -2.754   3.007
trend_pa_3     0.679 0.596   1.140 0.257  -0.503   1.861
trend_pa_5    -0.329 0.259  -1.271 0.207  -0.843   0.185
trend_pa_8     0.117 0.543   0.215 0.830  -0.960   1.193
trend_pa_15   -0.001 0.243  -0.004 0.997  -0.484   0.482
trend_pa_19    1.395 0.383   3.639 0.000   0.634   2.156
trend_pa_24    0.496 0.173   2.864 0.005   0.152   0.839
trend_pa_29    0.404 0.720   0.561 0.576  -1.024   1.832
trend_pa_1231  1.143 0.280   4.078 0.000   0.587   1.699
trend_pa_1690  1.343 0.300   4.481 0.000   0.748

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			1355
R-squared:		0.5867
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  120
Fixed effects by:	authority_code
  No. of FE:		  18
               coeff    se      t   p>t  CI_low CI_high
fpsb_auction  10.025 1.428  7.022 0.000   7.198  12.852
reserve_price  0.000 0.000  8.430 0.000   0.000   0.000
trend_pa_2    -0.084 1.491 -0.056 0.955  -3.037   2.869
trend_pa_3     1.071 0.640  1.672 0.097  -0.197   2.338
trend_pa_5    -0.630 0.237 -2.657 0.009  -1.100  -0.161
trend_pa_8     0.402 0.520  0.774 0.440  -0.627   1.432
trend_pa_15   -0.046 0.251 -0.182 0.856  -0.542   0.451
trend_pa_19    1.248 0.285  4.378 0.000   0.684   1.813
trend_pa_24    0.465 0.144  3.234 0.002   0.180   0.750
trend_pa_29    0.260 0.699  0.372 0.711  -1.124   1.644
trend_pa_33    1.043 0.197  5.300 0.000   0.653   1.432
trend_pa_585   0.788 0.510  1.544 0.125  -0.222   1.798
trend_pa_1231  1.039 0.269  3.864 0.000   0.506   1.571
trend

In [20]:
#table4 controls.do
#1st column
#treat 별로 outcome 확인하자
#turin_co_sample

#treatment = ['turin_co_sample','turin_pr_sample']
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration for group
for o in outcome:
    df_reg_co = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
    
    #vif cal
    #first, make a column list
    reg_col = ['trend_pa_2','trend_pa_3','trend_pa_5','trend_pa_8','trend_pa_15','trend_pa_19','trend_pa_24','trend_pa_29','trend_pa_1231',
              'trend_pa_1690','trend_pa_1721','trend_pa_1749','trend_pa_1839']
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','reserve_price','municipality','trend','trend_treat']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_reg_co.loc[:,exog]
    vif = calc_vif(X)
    print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')
    
    #exog.remove(2006)


    #print(exog) #check
    fe_reg = mt.reg(df_reg_co, o , exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(fe_reg)

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


        variables       VIF
0    fpsb_auction  7.409896
1   reserve_price  1.161375
2    municipality  6.868839
3           trend       inf
4     trend_treat  8.729106
..            ...       ...
66           2006       inf
67           2007       NaN
68           2008       NaN
69           2009       NaN
70           2010       NaN

[71 rows x 2 columns]
Dependent variable:	discount
N:			1262
R-squared:		0.6497
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  101
Fixed effects by:	authority_code
  No. of FE:		  15
               coeff    se       t   p>t  CI_low CI_high
fpsb_auction   5.941 1.381   4.302 0.000   3.201   8.681
reserve_price  0.000 0.000   6.110 0.000   0.000   0.000
trend_treat    2.264 0.416   5.440 0.000   1.438   3.089
trend_pa_2     0.126 1.452   0.087 0.931  -2.754   3.007
trend_pa_3     0.679 0.596   1.140 0.257  -0.503   1.861
trend_pa_5    -0.329 0.259  -1.271 0.207  -0.843   0.185
trend_pa_8     0.117 0.543   0.

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


        variables       VIF
0    fpsb_auction  7.409896
1   reserve_price  1.161375
2    municipality  6.868839
3           trend       inf
4     trend_treat  8.729106
..            ...       ...
66           2006       inf
67           2007       NaN
68           2008       NaN
69           2009       NaN
70           2010       NaN

[71 rows x 2 columns]
Dependent variable:	delay_ratio
N:			1110
R-squared:		0.1350
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  99
Fixed effects by:	authority_code
  No. of FE:		  15
                coeff     se      t   p>t  CI_low CI_high
fpsb_auction    4.765 27.970  0.170 0.865 -50.740  60.270
reserve_price  -0.000  0.000 -2.077 0.040  -0.000  -0.000
trend_treat     3.863  6.428  0.601 0.549  -8.893  16.619
trend_pa_2     -1.966 10.743 -0.183 0.855 -23.286  19.353
trend_pa_3     -4.715  6.831 -0.690 0.492 -18.271   8.841
trend_pa_5     16.196 14.979  1.081 0.282 -13.530  45.922
trend_pa_8    -10.182 

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


        variables       VIF
0    fpsb_auction  7.409896
1   reserve_price  1.161375
2    municipality  6.868839
3           trend       inf
4     trend_treat  8.729106
..            ...       ...
66           2006       inf
67           2007       NaN
68           2008       NaN
69           2009       NaN
70           2010       NaN

[71 rows x 2 columns]


LinAlgError: Singular matrix

In [ ]:
#table 4
#even columns
#2nd row: turin_pr_sample

#treatment = ['turin_co_sample','turin_pr_sample']
#co_sample이 잘 돌아가기 때문에 co부터
#treat 별로 outcome 확인하자
#pr도 돌려봄

outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration for group
for o in outcome:
    df_reg_co = df[(df['turin_pr_sample']==1)&(df['ctrl_exp_turin_pr_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
    
    #vif cal
    #first, make a column list
    reg_col = ['trend_pa_2','trend_pa_3','trend_pa_5','trend_pa_8','trend_pa_15','trend_pa_19','trend_pa_24','trend_pa_29','trend_pa_33','trend_pa_585','trend_pa_1231',
              'trend_pa_1480', 'trend_pa_1690','trend_pa_1721','trend_pa_1749','trend_pa_1839']
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','reserve_price','municipality','trend','trend_treat']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_reg_co.loc[:,exog]
    vif = calc_vif(X)
    print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')
    #exog.remove(2006)

    #print(exog) #check
    fe_reg = mt.reg(df_reg_co, o , exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(fe_reg)